In [1]:
from env import *
import random

/opt/conda/lib/python3.7/site-packages/stable_baselines3/common/cmd_util.py:6: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  "Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.", FutureWarning


In [2]:
### generate train environments ###
env_train_num = 100
opt_num = 10
sup_dim = 100
ob_num = 10
limit = np.array([10,20])
lr = 0.1

### generate hard train environments ###
random.seed(0)
env_train_hard_list1 = []
count = 0
while(count<env_train_num):
    env_try = generate_env(ob_num,limit,opt_num,sup_dim)
    x0 = env_try.obj.initial()
    free = False
    for j in range(200):
        x0 = x0 - lr*env_try.obj.ob_der_fun(x0)
    if not env_try.obj.collision(x0):
        env_train_hard_list1.append(env_try)
        count += 1
        if count % 10 == 0:
            print(count,'hard cases for training found')
    env_try.close()
env_train_hard_list = [make_env(env,i) for i,env in enumerate(env_train_hard_list1)]
env_train_hard = DummyVecEnv(env_train_hard_list)

### generate easy train environments ###
env_train_easy_list1 = []
for i in range(env_train_num):
    env_train_easy_list1.append(generate_env(ob_num,limit,opt_num,sup_dim))
env_train_easy_list = [make_env(env,i) for i,env in enumerate(env_train_easy_list1)]
env_train_easy = DummyVecEnv(env_train_easy_list)
### generate mixed train environments ###
env_train_mix_list1 = env_train_easy_list1[0:50] + env_train_hard_list1[0:50]
env_train_mix_list = [make_env(env,i) for i,env in enumerate(env_train_mix_list1)]
env_train_mix = DummyVecEnv(env_train_mix_list)

1 hard cases for training found
2 hard cases for training found
3 hard cases for training found
4 hard cases for training found
5 hard cases for training found
6 hard cases for training found
7 hard cases for training found
8 hard cases for training found
9 hard cases for training found
10 hard cases for training found
11 hard cases for training found
12 hard cases for training found
13 hard cases for training found
14 hard cases for training found
15 hard cases for training found
16 hard cases for training found
17 hard cases for training found
18 hard cases for training found
19 hard cases for training found
20 hard cases for training found
21 hard cases for training found
22 hard cases for training found
23 hard cases for training found
24 hard cases for training found
25 hard cases for training found
26 hard cases for training found
27 hard cases for training found
28 hard cases for training found
29 hard cases for training found
30 hard cases for training found
31 hard cases for t

In [3]:
### generate supervision data ###
# train the agent in easy/hard benchmark
env_train = env_train_easy

exp_data = env_train.env_method('supervision')
exp_obs = [i[0] for i in exp_data]
exp_act = [i[1] for i in exp_data]
exp_obs = np.concatenate(exp_obs).astype(np.float32)
exp_act = np.concatenate(exp_act).astype(np.float32)

exp_data = ExpertDataSet(exp_obs, exp_act)
train_size = int(0.8 * len(exp_data))
test_size = len(exp_data) - train_size
exp_train, exp_test = random_split(exp_data, [train_size, test_size])
### generate supervision data ###

In [4]:
train_new_model = True
if train_new_model:
    student = PPO(CustomActorCriticPolicy, env_train, n_steps = 40, gamma=1, verbose=1)
    pretrain_agent(
        student,
        exp_train = exp_train,
        exp_test = exp_test,
        epochs=1,
        scheduler_gamma=0.7,
        learning_rate=1.0,
        log_interval=1000,
        no_cuda=False,
        seed=1,
        batch_size=64)
    student.policy.float()
    student.learn(50000)
    student.save('obs_orig/easy_0.1_128_50k')
else:
    # load existing model
    student = PPO.load('obs_orig/mix_reward_0.1_PN64')

Using cuda device
Train Epoch: 1 [0/8000 (0%)]	Loss: 1.024513
Test set: Average loss: 0.0002
-----------------------------
| time/              |      |
|    fps             | 219  |
|    iterations      | 1    |
|    time_elapsed    | 18   |
|    total_timesteps | 4000 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 198        |
|    iterations           | 2          |
|    time_elapsed         | 40         |
|    total_timesteps      | 8000       |
| train/                  |            |
|    approx_kl            | 0.03441817 |
|    clip_fraction        | 0.288      |
|    clip_range           | 0.2        |
|    entropy_loss         | -20.7      |
|    explained_variance   | -1.98e+04  |
|    learning_rate        | 0.0003     |
|    loss                 | 3.12e+03   |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0321    |
|    std                  | 0.679      |


In [5]:
## runtime evaluation 
runtime_env_num = 100
ob_num = 10
limit = np.array([10,20])
opt_num = 10
sup_dim = 0
# generate run time evaluation environments
random.seed(1)
time_env_list1 = []
count = 0
while(count < runtime_env_num):
    done1 = False 
    done = False
    env_try = generate_env(ob_num,limit,opt_num,sup_dim)
    x0 = env_try.obj.initial()
    obs = env_try.reset()
    for j in range(100):
        x0 = x0 - 0.1*env_try.obj.ob_der_fun(x0)
        if env_try.obj.collision(x0):
            done1 = True
            break
    for k in range(100):
        action, _ = student.predict(obs, deterministic=True)
        obs, reward, done, info = env_try.step(action)
        if done:
            break
    if done1 and done and j > 10:
        time_env_list1.append(env_try)
        count += 1
        print(count,'run time evalution env found')
    env_try.close()

1 run time evalution env found
2 run time evalution env found
3 run time evalution env found
4 run time evalution env found
5 run time evalution env found
6 run time evalution env found
7 run time evalution env found
8 run time evalution env found
9 run time evalution env found
10 run time evalution env found
11 run time evalution env found
12 run time evalution env found
13 run time evalution env found
14 run time evalution env found
15 run time evalution env found
16 run time evalution env found
17 run time evalution env found
18 run time evalution env found
19 run time evalution env found
20 run time evalution env found
21 run time evalution env found
22 run time evalution env found
23 run time evalution env found
24 run time evalution env found
25 run time evalution env found
26 run time evalution env found
27 run time evalution env found
28 run time evalution env found
29 run time evalution env found
30 run time evalution env found
31 run time evalution env found
32 run time evalu

In [6]:
import time
env = time_env_list1[1]
x0 = env.obj.initial()
obs = env.reset()
ob_der_fun = env.obj.ob_der_fun
t1 = time.time()
for j in range(1000):
    x0 = x0 - lr*ob_der_fun(x0)
t_gd = time.time()-t1
t1 = time.time()
for j in range(1000):
    action, _ = student.predict(obs, deterministic=True)
    env.pos = action.reshape(10,2) + env.pos
    obs = env.obj.ob_der_fun(env.pos).flatten()
t_rl = time.time()-t1
print(t_gd,t_rl)
runtime(student, time_env_list1)

1.214087724685669 1.8839995861053467

GD success rate: 1.0 GD average iterations= 26.56
RL success rate: 1.0 RL average iterations= 18.04



In [7]:
# successful rate evaluation
env_test_easy_num = 1000
env_test_hard_num = 100

# generate easy test environments
random.seed(2)
env_test_easy_list1 = []
for i in range(env_test_easy_num):
    env_test_easy_list1.append(generate_env(ob_num,limit,opt_num,0))
# generate hard test environments
env_test_hard_list1 = []
count = 0
while(count<env_test_hard_num):
    env_try = generate_env(ob_num,limit,opt_num,sup_dim)
    x0 = env_try.obj.initial()
    free = False
    for j in range(200):
        x0 = x0 - lr*env_try.obj.ob_der_fun(x0)
        if env_try.obj.collision(x0):
            free = True
            break
    if not free:
        env_test_hard_list1.append(env_try)
        count += 1
        print(count,'hard cases for test found')
    env_try.close()

1 hard cases for test found
2 hard cases for test found
3 hard cases for test found
4 hard cases for test found
5 hard cases for test found
6 hard cases for test found
7 hard cases for test found
8 hard cases for test found
9 hard cases for test found
10 hard cases for test found
11 hard cases for test found
12 hard cases for test found
13 hard cases for test found
14 hard cases for test found
15 hard cases for test found
16 hard cases for test found
17 hard cases for test found
18 hard cases for test found
19 hard cases for test found
20 hard cases for test found
21 hard cases for test found
22 hard cases for test found
23 hard cases for test found
24 hard cases for test found
25 hard cases for test found
26 hard cases for test found
27 hard cases for test found
28 hard cases for test found
29 hard cases for test found
30 hard cases for test found
31 hard cases for test found
32 hard cases for test found
33 hard cases for test found
34 hard cases for test found
35 hard cases for test 

In [8]:
### easy test benchmark ###
n_steps = 200
lr = 0.1
result_easy = np.zeros((4,))
i = 0
for env_test in env_test_easy_list1:
    obs = env_test.reset()
    x0 = env_test.pos
    for step in range(n_steps):
        x0 = x0 - lr*env_test.obj.ob_der_fun(x0)
    for step in range(n_steps):
        action, _ = student.predict(obs, deterministic=True)
        obs, reward, done, info = env_test.step(action)
        if done:
            #print("Goal reached!", "reward=", reward,"step=",step)
            break
    if done and env_test.obj.collision(x0):
        result_easy[0] += 1
    if not done and not env_test.obj.collision(x0):
        result_easy[1] += 1
    if done and not env_test.obj.collision(x0):
        result_easy[2] += 1
    if not done and env_test.obj.collision(x0):
        result_easy[3] += 1
    env_test.close()
    if (i+1) % 50 == 0:
        print((i+1)/len(env_test_easy_list1),'complete')
    i += 1
result_easy /= len(env_test_easy_list1)
print("result_list_easy:", result_easy)
rl_success = result_easy[0]+result_easy[2]
gd_success = result_easy[0]+result_easy[3]
print('success_rl: %.2f%%'  % (rl_success*100))
print('success_gd: %.2f%%'  % (gd_success*100))

0.05 complete
0.1 complete
0.15 complete
0.2 complete
0.25 complete
0.3 complete
0.35 complete
0.4 complete
0.45 complete
0.5 complete
0.55 complete
0.6 complete
0.65 complete
0.7 complete
0.75 complete
0.8 complete
0.85 complete
0.9 complete
0.95 complete
1.0 complete
result_list_easy: [0.737 0.169 0.075 0.019]
success_rl: 81.20%
success_gd: 75.60%


In [9]:
### hard test benchmark ###
n_steps = 200
lr = 0.1
result_hard_GD = 0
result_hard_RL = 0
for env_test in env_test_hard_list1:
    obs = env_test.reset()
    x0 = env_test.pos
    for step in range(n_steps):
        x0 = x0 - lr * env_test.obj.ob_der_fun(x0)
        if env_test.obj.collision(x0):
            result_hard_GD += 1
            break
            
    for step in range(n_steps):
        action, _ = student.predict(obs, deterministic=True)
        obs, reward, done, info = env_test.step(action)
        if done:
            result_hard_RL += 1
            break
    env_test.close()
result_hard_GD /= len(env_test_hard_list1)
result_hard_RL /= len(env_test_hard_list1)
print("result_list_hard_GD", result_hard_GD)
print("result_list_hard_RL", result_hard_RL)

result_list_hard_GD 0.0
result_list_hard_RL 0.23


In [10]:
### multi start on easy test benchmark
multi_start_num = 5
result_multi_start = np.zeros((4,))

count = 0
for env_test in env_test_easy_list1:
    GD = False
    RL = False
    i=0
    while(GD==False and i<multi_start_num):
        if i == 0:
            obs = env_test.reset()
        else:
            obs = env_test.reset(random_start = True)
        x0 = env_test.pos
        for step in range(n_steps):
            x0 = x0 - lr*env_test.obj.ob_der_fun(x0)
            if env_test.obj.collision(x0):
                GD = True
                # print('GD finds feasible solution with',i,'trials on',count,'th environment')
                break
        i += 1
    i=0
    while(RL==False and i<multi_start_num):
        if i == 0:
            obs = env_test.reset()
        else:
            obs = env_test.reset(random_start = True)
        x0 = env_test.pos
        for step in range(n_steps):
            action, _ = student.predict(obs, deterministic=True)
            obs, reward, done, info = env_test.step(action)
            if done:
                RL = True
                # print('RL finds feasible solution with',i,'trials on',count,'th environment')
                break
        i += 1

    if GD == True and RL == True:
        result_multi_start[0] += 1
    elif GD == False and RL == False:
        result_multi_start[1] += 1
    elif GD == False and RL == True:
        result_multi_start[2] += 1
    else: 
        result_multi_start[3] += 1
    if (count+1) % 50 == 0:
        print((count+1)/len(env_test_easy_list1),'complete')
    count += 1
    env_test.close()
    
result_multi_start /= len(env_test_easy_list1)
print("result_list_easy_ms:", result_multi_start)
rl_success_multi = result_multi_start[0]+result_multi_start[2]
gd_success_multi = result_multi_start[0]+result_multi_start[3]
print('success_rl: %.2f%% ' % (rl_success_multi*100))
print('success_gd: %.2f%%'  % (gd_success_multi*100))

0.05 complete


KeyboardInterrupt: 

In [ ]:
### multi start on hard test benchmark ###
multi_start_num = 5
result_multi_start = np.zeros((4,))

count = 0
for env_test in env_test_hard_list1:
    GD = False
    RL = False
    i=0
    while(GD==False and i<multi_start_num):
        if i == 0:
            obs = env_test.reset()
        else:
            obs = env_test.reset(random_start = True)
        x0 = env_test.pos
        for step in range(n_steps):
            x0 = x0 - lr*env_test.obj.ob_der_fun(x0)
            if env_test.obj.collision(x0):
                GD = True
                break
        i += 1
    i=0
    while(RL==False and i<multi_start_num):
        if i ==0:
            obs = env_test.reset()
        else:
            obs = env_test.reset(random_start = True)
        x0 = env_test.pos
        for step in range(n_steps):
            action, _ = student.predict(obs, deterministic=True)
            obs, reward, done, info = env_test.step(action)
            if done:
                RL = True
                break
        i += 1

    if GD == True and RL == True:
        result_multi_start[0] += 1
    elif GD == False and RL == False:
        result_multi_start[1] += 1
    elif GD == False and RL == True:
        result_multi_start[2] += 1
    else: 
        result_multi_start[3] += 1
    if (count+1) % 50 == 0:
        print((count+1)/len(env_test_hard_list1),'complete')
    count += 1
    env_test.close()
    
result_multi_start /= len(env_test_hard_list1)
print("result_list_hard_ms:", result_multi_start)
rl_success_multi = result_multi_start[0]+result_multi_start[2]
gd_success_multi = result_multi_start[0]+result_multi_start[3]
print('success_rl: %.2f%% ' % (rl_success_multi*100))
print('success_gd: %.2f%%'  % (gd_success_multi*100))